In [1]:
import dask
from dask.distributed import Client, LocalCluster, get_worker
from dask.distributed.diagnostics.plugin import WorkerPlugin
import fsspec
import logging

In [2]:
def complex():
    get_worker().log_event("complex", {"hard": "hard", "worker_id": get_worker().id})
    logging.info(get_worker().id)
    return 'hard'


class DaskWorkerHandler(WorkerPlugin):
    """Worker life-cycle handler."""
    def __init__(self):
        self.worker_id = None

    def setup(self, worker):
        self.worker_id = worker.id

    def teardown(self, worker):
        logs = worker.logs
        with fsspec.open("worker-logs.txt", "w") as f:
            f.write("\n".join([str(log) for log in logs]))


if __name__ == '__main__':
    dask.config.set({'logging.distributed': "INFO"})
    
    cluster = LocalCluster(n_workers=5)
    client = Client(cluster)

    shutdown_handler = DaskWorkerHandler()
    client.register_worker_plugin(shutdown_handler)

    future = client.submit(complex)
    events = client.get_events('complex')
    print(events)
    result = future.result()
    
    client.close()
    cluster.close()

()
